In [2]:
#logistic classification을 diabetes data를 활용해서 모델을 만들어보자
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()
tf.set_random_seed(777)
print(tf.__version__)


# data
xy = np.loadtxt('data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_train = xy[:, 0:-1]
y_train = xy[:, [-1]]

print('xy:\n{}'.format(xy))
print('x_train:\n{}'.format(x_train))
# print('y_train:\n{}'.format(y_train))
print(x_train.shape, y_train.shape)

# initialize dataset, W, b
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))
W = tf.Variable(tf.random_normal([8, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# hypothesis로는 WX+b를 Sigmoid activation을 취한 값을 사용
# Sigmoid: 1 / (1 + exp(Wx+b))
# z = Wx+b, g(z), g(z) = 1/(1+exp(z))
def logistic_regression(features):
  hypothesis = tf.div(1., 1. + tf.exp(tf.matmul(features, W) + b))
  return hypothesis

# 손실 함수로는 crossentropy 함수를 사용한다.
# crossentropy함수는 y * log(hypothesis + (1-y)*log(1-hypothesis))
def loss_fn(features, labels):
  hypothesis = logistic_regression(features)
  loss = -tf.reduce_mean(labels * tf.log(logistic_regression(features)) + (1-labels) * tf.log(1 - hypothesis))
  return loss

# 손실 값의 gradient를 계산한다.
def grad(features, labels):
  hypothesis = logistic_regression(features)
  with tf.GradientTape() as tape:
    loss_value = loss_fn(features, labels)
  return tape.gradient(loss_value, [W, b])

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

# 1000번의 epoch동안 gradient를 변화시켜 가며 손실이 최소가 되는 점을 찾는다
EPOCHS = 1001
for step in range(EPOCHS):
  for features, labels in dataset:
    grads = grad(features, labels)
    optimizer.apply_gradients(grads_and_vars=zip(grads, [W, b]))
    if step % 100 == 0:
      print('Iter: {}, Loss: {:.4f}'.format(step, loss_fn(features, labels)))

1.13.1
xy:
[[-0.294118   0.487437   0.180328  ... -0.53117   -0.0333333  0.       ]
 [-0.882353  -0.145729   0.0819672 ... -0.766866  -0.666667   1.       ]
 [-0.0588235  0.839196   0.0491803 ... -0.492741  -0.633333   0.       ]
 ...
 [-0.411765   0.21608    0.180328  ... -0.857387  -0.7        1.       ]
 [-0.882353   0.266332  -0.0163934 ... -0.768574  -0.133333   0.       ]
 [-0.882353  -0.0653266  0.147541  ... -0.797609  -0.933333   1.       ]]
x_train:
[[-0.294118    0.487437    0.180328   ...  0.00149028 -0.53117
  -0.0333333 ]
 [-0.882353   -0.145729    0.0819672  ... -0.207153   -0.766866
  -0.666667  ]
 [-0.0588235   0.839196    0.0491803  ... -0.305514   -0.492741
  -0.633333  ]
 ...
 [-0.411765    0.21608     0.180328   ... -0.219076   -0.857387
  -0.7       ]
 [-0.882353    0.266332   -0.0163934  ... -0.102832   -0.768574
  -0.133333  ]
 [-0.882353   -0.0653266   0.147541   ... -0.0938897  -0.797609
  -0.933333  ]]
(759, 8) (759, 1)
Iter: 0, Loss: 0.6556
Iter: 100, Loss: 